In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-14"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
30233,2024-12-14,Portugal Lpb,08:45,Sporting CP,Galomar,1.04,7.50,158.5,1.83,1.83,-16.5,1.98,1.70,n9YCZHu9,0.961538,0.133333,0.546448,0.546448,0.094872,107.594,25.229968,0.234492,1.8,1.643168,0.912871,88.40,1.228,0.242837,0.197750,59.0,409.230,371.066654,0.906744,0.6,1.341641,2.236068,411.18,5.268,4.457014,0.846054,-27.0,68,66,1.30,6.23,101.844,380.638,1.069768,0.000000,0.107603,-0.67,-0.134,-0.298507,0.888237,0.6,-0.288237,-2.53,-0.506,-12.845850,0.000000,0.0,0.000000
30234,2024-12-14,Polônia 1. Liga,11:00,Trefl Sopot II,HydroTruck Radom,1.78,1.89,165.5,1.83,1.83,-2.5,2.01,1.68,hMB2Nsbt,0.561798,0.529101,0.546448,0.546448,0.090898,195.932,64.791008,0.330681,0.6,1.341641,2.236068,154.80,2.326,0.699736,0.300832,-26.0,227.052,49.303096,0.217145,0.6,1.341641,2.236068,185.84,2.842,0.622109,0.218898,-35.0,72,92,2.15,2.02,145.614,233.484,0.042388,0.000000,0.126474,-1.19,-0.238,-3.277311,0.000000,0.0,0.000000,-5.00,-1.000,-0.890000,0.381703,0.3,-0.081703
30235,2024-12-14,Polônia Liga Feminina De Basquete,12:00,Gorzow F,Polkowice F,1.52,2.30,147.5,1.83,1.83,-5.5,2.05,1.65,6F0SJI13,0.657895,0.434783,0.546448,0.546448,0.092677,144.398,54.272710,0.375855,1.2,1.643168,1.369306,209.60,1.870,0.658445,0.352110,28.0,112.454,32.845942,0.292083,3.0,0.000000,0.000000,159.36,1.328,0.407517,0.306865,39.0,80,83,2.62,1.92,114.400,100.466,0.288766,0.000000,0.152888,-1.28,-0.256,-2.031250,0.775863,0.7,-0.075863,-0.19,-0.038,-34.210526,0.775828,0.8,0.024172
30236,2024-12-14,Polônia 1. Liga,16:00,Poznań,Poli. Opolska,1.44,2.52,164.5,1.83,1.83,-7.5,1.95,1.72,z53JJac5,0.694444,0.396825,0.546448,0.546448,0.091270,135.788,30.855565,0.227233,1.8,1.643168,0.912871,181.44,1.526,0.362395,0.237480,7.0,133.164,14.230730,0.106866,0.6,1.341641,2.236068,143.64,1.672,0.163003,0.097490,-28.0,96,76,1.89,1.89,119.130,149.986,0.385695,0.000000,0.088629,-1.07,-0.214,-2.056075,0.755136,0.7,-0.055136,0.54,0.108,14.074074,0.525171,0.7,0.174829
30237,2024-12-14,Polônia 1. Liga,16:00,Krosno,WKK Wroclaw,1.45,2.65,162.5,1.83,1.83,-8.5,2.01,1.68,vwAALLSh,0.689655,0.377358,0.546448,0.546448,0.067014,147.210,24.798411,0.168456,2.4,1.341641,0.559017,137.06,1.752,0.397706,0.227001,9.0,195.436,38.698177,0.198009,2.4,1.341641,0.559017,193.44,2.390,0.639336,0.267504,31.0,77,78,1.78,2.48,118.072,197.536,0.413916,0.000000,0.126474,-2.10,-0.420,-1.071429,0.747101,0.7,-0.047101,1.67,0.334,4.940120,0.359866,0.5,0.140134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30569,2024-12-14,Eua Ncaa,18:00,Pepperdine,Northern Arizona,1.51,2.61,144.5,1.83,1.83,-4.5,1.83,1.83,WYB5z12e,0.662252,0.383142,0.546448,0.546448,0.045393,241.590,216.844707,0.897573,1.2,1.643168,1.369306,150.45,3.750,3.802335,1.013956,-5.0,143.364,41.065425,0.286442,1.8,1.643168,0.912871,194.75,1.832,0.381995,0.208512,6.0,85,95,1.77,2.05,142.484,305.484,0.377581,0.000000,0.000000,1.12,0.224,2.276786,0.527328,0.5,-0.027328,-3.55,-0.710,-2.267606,0.350294,0.4,0.049706
30570,2024-12-14,Eua Ncaa,18:00,North Carolina,La Salle,1.02,13.50,163.5,1.83,1.83,-20.5,1.91,1.80,dKTBLzJs,0.980392,0.074074,0.546448,0.546448,0.054466,118.

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
30234,11:00,Polônia 1. Liga,Trefl Sopot II,HydroTruck Radom,1.78,Back Home
30241,10:00,Noruega Blno,Nidaros Jets,Asker,1.74,Back Home
30267,15:00,Portugal Proliga,Portimonense,Lusitania,2.09,Back Home
30275,03:05,Japão B.League,Ibaraki Robots,Sendai,1.56,Back Home
30356,20:00,Eua Ncaa,Niagara,St. Francis (PA),1.58,Back Home
30364,15:30,Sérvia 1. Zls Feminino,Crvena zvezda F,Novosadska Novi Sad F,1.83,Back Home
30409,14:00,Croácia Premijer Liga,Zabok,Dinamo Zagreb,1.73,Back Home
30419,08:35,China Cba,Guangzhou,Qingdao,2.77,Back Home
30437,15:00,Eslovênia Liga Otp Banka,Lasko,Rogaska,1.80,Back Home
30446,03:00,Cazaquistão Higher League,Turan,Karaganda,1.83,Back Home
